# TPC3: Melhoramento de uma versão de TPC2 de um grupo 

### Codigo base

In [39]:
from lark import Lark, Transformer, v_args

grammar = """
start: signal intervals "."       -> sentence

signal: "+"                    -> plus_signal
      | "-"                    -> minus_signal

intervals: interval remaining_intervals -> intervals
remaining_intervals:          -> empty
                   | interval remaining_intervals -> remaining_intervals
interval: "[" NUMBER ";" NUMBER "]"    -> interval

%import common.NUMBER
%import common.WS_INLINE
%ignore WS_INLINE
"""

grammar2 = """
start: signal intervals "."

signal: "+"
      | "-"

intervals: interval remaining_intervals
remaining_intervals:
                   | interval remaining_intervals
interval: "[" NUMBER ";" NUMBER "]"

%import common.NUMBER
%import common.WS_INLINE
%ignore WS_INLINE
"""

class IntervalTransformer(Transformer):
    """
    Handles transformations and applies constraints to parsed intervals.

    Attributes:
        sentido (int): Indicates the current signal direction (+1 or -1).
        anterior (float): Tracks the end value of the previous interval.
        erro (bool): Flag indicating if a constraint was violated.
    """
    def __init__(self):
        self.sentido = 0
        self.anterior = float('-inf')
        self.erro = False

    @v_args(inline=True)
    def sentence(self, signal, intervals):
        """Returns the parsed intervals."""
        return intervals  # !TODO return a dict with is_valid and then the struct of data


    @v_args(inline=True)
    def plus_signal(self):
        """Sets the signal direction to +1 (growing)."""
        self.sentido = 1
        print("Signal set to + (growing).")
        return self.sentido

    @v_args(inline=True)
    def minus_signal(self):
        """Sets the signal direction to -1 (decreasing)."""
        self.sentido = -1
        self.anterior = float('inf')
        print("Signal set to - (decreasing).")
        return self.sentido

    @v_args(inline=True)
    def interval(self, start, end):
        """
        Processes an interval and checks constraints based on signal direction.
        !TODO Add a function for the remaining intervals
        """
        start = int(start)
        end = int(end)

        if self.sentido == 1:
            if end <= start:
                print(f"Error: Constraint CC1 violated. End ({end}) must be greater than Start ({start}).")
                self.erro = True
            if start < self.anterior:
                print(f"Error: Constraint CC2 violated. Start ({start}) must be greater than or equal to the previous End ({self.anterior}).")
                self.erro = True
        elif self.sentido == -1:
            if end >= start:
                print(f"Error: Constraint CC1 violated. End ({end}) must be less than Start ({start}).")
                self.erro = True
            if start > self.anterior:
                print(f"Error: Constraint CC2 violated. Start ({start}) must be less than or equal to the previous End ({self.anterior}).")
                self.erro = True

        self.anterior = end
        return (start, end)

    def empty(self, _):
        """Represents an empty remaining interval."""
        return []

parser = Lark(grammar, parser='lalr', transformer=None)

def parse_input(input_text):
    """
    Parses the input text, applies transformations, and checks constraints.

    Args:
        input_text (str): The input string to parse.

    Returns:
        None
    """
    try:
        tree = parser.parse(input_text)
        transformer = IntervalTransformer()
        result = transformer.transform(tree)
        if transformer.erro:
            print("\nParsing failed: One or more constraints were violated.")
        else:
            print("\nParsing successful: All constraints satisfied.")
            print("Parsed intervals:", result)
    except Exception as e:
        print(f"\nParsing error: {e}")

if __name__ == "__main__":
    input_text = "+ [1;3] [3;5] [6;8] ."
    print("Input:", input_text)
    parse_input(input_text)

    input_text = "- [8;6] [6;4] [4;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)

    input_text = "+ [1;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)

    input_text = "- [1;3] [2;4] [5;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)


"""
Input: + [1;3] [3;5] [6;8] .
sentence
  plus_signal
  intervals
    interval
      1
      3
    remaining_intervals
      interval
        3
        5
      remaining_intervals
        interval
          6
          8
        empty
"""

Input: + [1;3] [3;5] [6;8] .
Signal set to + (growing).

Parsing successful: All constraints satisfied.
Parsed intervals: Tree('intervals', [(1, 3), Tree('remaining_intervals', [(3, 5), Tree('remaining_intervals', [(6, 8), []])])])

Input: - [8;6] [6;4] [4;2] .
Signal set to - (decreasing).

Parsing successful: All constraints satisfied.
Parsed intervals: Tree('intervals', [(8, 6), Tree('remaining_intervals', [(6, 4), Tree('remaining_intervals', [(4, 2), []])])])

Input: + [1;2] .
Signal set to + (growing).

Parsing successful: All constraints satisfied.
Parsed intervals: Tree('intervals', [(1, 2), []])

Input: - [1;3] [2;4] [5;2] .
Signal set to - (decreasing).
Error: Constraint CC1 violated. End (3) must be less than Start (1).
Error: Constraint CC1 violated. End (4) must be less than Start (2).
Error: Constraint CC2 violated. Start (5) must be less than or equal to the previous End (4).

Parsing failed: One or more constraints were violated.


'\nInput: + [1;3] [3;5] [6;8] .\nsentence\n  plus_signal\n  intervals\n    interval\n      1\n      3\n    remaining_intervals\n      interval\n        3\n        5\n      remaining_intervals\n        interval\n          6\n          8\n        empty\n'

Devemos portanto alterar o codigo de base de forma a retornar um dicionario com a perceção se os intervalos fornecidos são validos e qual é a maior amplitude entre os intervalos

## Versão Corregida

In [38]:
from lark import Lark, Transformer, v_args


grammar2 = """
start: signal intervals "."

signal: "+"
      | "-"

intervals: interval remaining_intervals
remaining_intervals:
                   | interval remaining_intervals
interval: "[" NUMBER ";" NUMBER "]"

%import common.NUMBER
%import common.WS_INLINE
%ignore WS_INLINE
"""

class IntervalTransformer(Transformer):
    """
    Handles transformations and applies constraints to parsed intervals.

    Attributes:
        sentido (int): Indicates the current signal direction (+1 or -1).
        anterior (float): Tracks the end value of the previous interval.
        erro (bool): Flag indicating if a constraint was violated.
    """
    def __init__(self):
        self.sentido = 0
        self.anterior = float('-inf')
        self.erro = False
        self.isvalid = True 
        self.diferenca = 0 
        self.intra = []
        self.data_struct = {
            "isvalid": self.isvalid,
            "diferenca": self.diferenca
        }


    @v_args(inline=True)
    def sentence(self, signal, intervals):
        """Verifica se os intervalos seguem a ordem correta de acordo com o sinal."""
        self.isvalid = not  (self.erro) 
        if self.isvalid: 
            self.intra.sort()
            self.diferenca = self.intra[-1] - self.intra[0]

        self.data_struct["isvalid"] = self.isvalid
        self.data_struct["diferenca"] = self.diferenca
        
        if self.isvalid: 
            print ( self.data_struct)

        return intervals


    @v_args(inline=True)
    def plus_signal(self):
        """Sets the signal direction to +1 (growing)."""
        self.sentido = 1
        return self.sentido

    @v_args(inline=True)
    def minus_signal(self):
        """Sets the signal direction to -1 (decreasing)."""
        self.sentido = -1
        self.anterior = float('inf')
        return self.sentido

    @v_args(inline=True)
    def interval(self, start, end):
        """
        Processes an interval and checks constraints based on signal direction.
        """
        start = int(start)
        end = int(end)

        if self.sentido == 1:
            if end <= start:
                print(f"Error: Constraint CC1 violated. End ({end}) must be greater than Start ({start}).")
                self.erro = True
            if start < self.anterior:
                print(f"Error: Constraint CC2 violated. Start ({start}) must be greater than or equal to the previous End ({self.anterior}).")
                self.erro = True
        elif self.sentido == -1:
            if end >= start:
                print(f"Error: Constraint CC1 violated. End ({end}) must be less than Start ({start}).")
                self.erro = True
            if start > self.anterior:
                print(f"Error: Constraint CC2 violated. Start ({start}) must be less than or equal to the previous End ({self.anterior}).")
                self.erro = True


        self.anterior = end
        self.intra.extend([start,end])

        return (start, end)

    def empty(self, _):
        """Represents an empty remaining interval."""
        return []

parser = Lark(grammar, parser='lalr', transformer=None)

def parse_input(input_text):
    """
    Parses the input text, applies transformations, and checks constraints.

    Args:
        input_text (str): The input string to parse.

    Returns:
        None
    """
    try:
        tree = parser.parse(input_text)
        transformer = IntervalTransformer()
        result = transformer.transform(tree)
        if transformer.erro:
            print("Parsing failed: One or more constraints were violated.")
        else:
            pass
    except Exception as e:
        print(f"\nParsing error: {e}")

if __name__ == "__main__":
    input_text = "+ [1;3] [3;5] [6;8] ."
    print("Input:", input_text)
    parse_input(input_text)

    input_text = "- [8;6] [6;4] [4;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)

    input_text = "+ [1;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)

    input_text = "- [1;3] [2;4] [5;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)


    input_text = "- [3;1] [4;2] [10;2] ."
    print("\nInput:", input_text)
    parse_input(input_text)


Input: + [1;3] [3;5] [6;8] .
{'isvalid': True, 'diferenca': 7}

Input: - [8;6] [6;4] [4;2] .
{'isvalid': True, 'diferenca': 6}

Input: + [1;2] .
{'isvalid': True, 'diferenca': 1}

Input: - [1;3] [2;4] [5;2] .
Error: Constraint CC1 violated. End (3) must be less than Start (1).
Error: Constraint CC1 violated. End (4) must be less than Start (2).
Error: Constraint CC2 violated. Start (5) must be less than or equal to the previous End (4).
Parsing failed: One or more constraints were violated.

Input: - [3;1] [4;2] [10;2] .
Error: Constraint CC2 violated. Start (4) must be less than or equal to the previous End (1).
Error: Constraint CC2 violated. Start (10) must be less than or equal to the previous End (2).
Parsing failed: One or more constraints were violated.


Objetivos da melhoria: 

1. adicionar um ***isvalid***, que confirma se os intervalos do nosso input estao de acordo com o sinal do mesmo 
    
2. calcular a difença entre o primeiro numero do intervalo e do ultimo. 

Para o nosso primeiro objetivo, percebemos que o codigo base já tinha defino o varivel de contexto ***erro** que confirmava as regras do intervalo estar corretamente ordenado. 

Para o segundo objetivo dicionamos criar uma estrutura de dados, uma lista, onde iamos acrescentando todos os elementos validados nas regras de ***intervals***. Sendo assim facil realizar os calculos da diferença. 